In [13]:
from FNN import FNN
from Layer import Layer
from Neuron import Neuron
import numpy as np
from ActivFunctions import  *
from InitFunctions import  *
from SuppFunctions import  *
from ErrorClasses import *

Neuron Class functionality testing

In [14]:
inputTest = np.array([1.0,1.0])

In [15]:
#zero initialization
t0NeuronZero = Neuron(2,identity)
print(t0NeuronZero.weight_vector.shape)
t0NeuronZero.forward(inputTest)

(1, 3)


[array([[0.]]), array([[0.]])]

In [16]:
#random initialization
t0NeuronRandom = Neuron(2,identity,method_ini="Random")
print(t0NeuronRandom.weight_vector.shape)
t0NeuronRandom.forward(inputTest)

(1, 3)


[array([[1.46474716]]), array([[1.46474716]])]

In [17]:
#initialization from list
vector = [1.0,2.0,3.5]
print(vector)
tNeuron = Neuron(vector,identity)
print(tNeuron.weight_vector.shape)
tNeuron.forward(inputTest)

[1.0, 2.0, 3.5]
(1, 3)


[array([[6.5]]), array([[6.5]])]

In [18]:
#initialization from vector
vector = np.array([1.0,2.0,3.5])
print(vector)
print(vector.shape)
t1Neuron = Neuron(vector,identity)
print(t1Neuron.weight_vector.shape)
t1Neuron.forward(inputTest)

[1.  2.  3.5]
(3,)
(1, 3)


[array([[6.5]]), array([[6.5]])]

In [19]:
#initialization from row vector(2D array)
vector_row = vector[np.newaxis,:]
print(vector_row)
print(vector_row.shape)
t2Neuron = Neuron(vector_row,identity)
print(t2Neuron.weight_vector.shape)
t2Neuron.forward(inputTest)

[[1.  2.  3.5]]
(1, 3)
(1, 3)


[array([[6.5]]), array([[6.5]])]

In [20]:
#initialization from column vector(2D array)
vector_column = vector[:,np.newaxis]
print(vector_column)
print(vector_column.shape)
t3Neuron = Neuron(vector_column,identity)
print(t3Neuron.weight_vector.shape)
t3Neuron.forward(inputTest)

[[1. ]
 [2. ]
 [3.5]]
(3, 1)
(1, 3)


[array([[6.5]]), array([[6.5]])]

In [21]:
#array input
t0NeuronRandom = Neuron(2,identity,method_ini="Random")
inputTest = np.array([[1.0,1.0],[1.0,1.0]])
print(inputTest.shape)
print(inputTest)
print(t0NeuronRandom.weight_vector.shape)
t0NeuronRandom.forward(inputTest)

(2, 2)
[[1. 1.]
 [1. 1.]]
(1, 3)


[array([[1.75911871, 1.75911871]]), array([[1.75911871, 1.75911871]])]

In [22]:
#list input
vector = np.array([1.0,2.0,3.5])
t0NeuronRandom = Neuron(vector,identity,method_ini="Random")
inputTest = ["1.0","1.0"]
t0NeuronRandom.forward(inputTest)

[array([[6.5]]), array([[6.5]])]

In [23]:
#error messages testing
print("Test for: no function given as activation function")
try:
    vector = np.array([1.0,2.0,3.5])
    errorNeuron = Neuron(vector,1)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (2 dim)")
try:
    vector = np.array([[1.0,2.0,3.5],[1.0,2.0,3.5]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (higher than 2 dim)")
try:
    vector = np.array([[[1.0,2.0,3.5],[1.0,2.0,3.5]],[[1.0,2.0,3.5],[1.0,2.0,3.5]]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong weights given for initialization")
try:
    vector = np.array(["1.0","2.0","3.5"])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given in list for weight initialization")
try:
    vector = ["as","2.0","3.5"]
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - inproper dimension of input")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="Random")
    inputTest = np.array([[[1.0,1.0],[1.0,1.0]],[[1.0,1.0],[1.0,1.0]]]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - wrong value types in input: they are not rational numbers")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="Random")
    inputTest = np.array([1.0 + 3j,1.0]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type for list")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="Random")
    inputTest = ["a","1.0"]
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="Random")
    inputTest = "a"
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too small)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorSmall = np.array([1.0])
    neuronTest.forward(inputTestErrorSmall)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too big)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorBig = np.array([[1.0],[1.0],[1.0],[1.0]])
    neuronTest.forward(inputTestErrorBig)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: no function given as activation function
Correct errors was caught. Error: Given input is not supported by implementation for activation function initialization. Given variable is not a function and thus can not be used as activation function.
Test for: not vector given for weights initialization (2 dim)
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given variable is not an vector (not column or row vector in 2 dim), so it is unsuitable for neuron weight initialization.
Test for: not vector given for weights initialization (higher than 2 dim)
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization. Given variable is not an vector (due to more than 2 dim), so it is unsuitable for neuron weight initialization.
Test for: wrong weights given for initialization
Correct errors was caught. Error: Given input is not supported by implementation for weights initialization.

Layer class implementation testing

In [24]:
#initialization by number of dim in vector
dim_layer = np.array([4,5])
testLayer = Layer(dim_layer,identity,method_ini="Random")
testLayer.weights_array
testLayer.activ_functions

[<function ActivFunctions.identity(x)>,
 <function ActivFunctions.identity(x)>,
 <function ActivFunctions.identity(x)>,
 <function ActivFunctions.identity(x)>]

In [25]:
#forward test pass
inputTestVect = np.array([[1.0],[1.0],[1.0],[1.0],[1.0]])
inputTestArray = np.array([[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]])

print(testLayer.forward(inputTestVect))
print(testLayer.forward(inputTestArray))

[array([[3.52236874],
       [1.87185115],
       [1.79746144],
       [2.0958627 ]]), array([[3.52236874],
       [1.87185115],
       [1.79746144],
       [2.0958627 ]])]
[array([[3.52236874, 6.60333582],
       [1.87185115, 3.70941041],
       [1.79746144, 3.18514732],
       [2.0958627 , 3.99410648]]), array([[3.52236874, 6.60333582],
       [1.87185115, 3.70941041],
       [1.79746144, 3.18514732],
       [2.0958627 , 3.99410648]])]


FNN class implementation testing

In [26]:
#initialization by number of dim in vector
dim_layer = np.array([5,5,3,2,1])
testNet = FNN(dim_layer,identity,method_ini="Random")
print(testNet.weights_list[0])
print(testNet.weights_list[1])
print(testNet.weights_list[2])
print(testNet.weights_list[3])
print(testNet.activ_functions_list[0])
print(testNet.activ_functions_list[1])
print(testNet.activ_functions_list[2])
print(testNet.activ_functions_list[3])
#forward test pass
inputTestVect = np.array([[1.0],[1.0],[1.0],[1.0],[1.0]])
inputTestArray = np.array([[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]])

print(testNet.forward(inputTestVect))
print(testNet.forward(inputTestArray))


[[0.66152791 0.9198933  0.44023579 0.98785632 0.08729518 0.40834576]
 [0.39880956 0.02394781 0.16799403 0.0795062  0.95861091 0.86216137]
 [0.84393168 0.7566142  0.69575119 0.33000233 0.04213909 0.46946175]
 [0.70011599 0.57439614 0.09043425 0.70993376 0.43419906 0.40226633]
 [0.86652418 0.57965949 0.56135131 0.02144302 0.80796942 0.95275965]]
[[0.31752958 0.22597783 0.32789189 0.58902019 0.59320987 0.26989915]
 [0.2806045  0.05000154 0.8557056  0.71420154 0.22869873 0.37313043]
 [0.24509897 0.04101323 0.67290989 0.08941846 0.20245231 0.04030879]]
[[0.52025242 0.02986111 0.65247902 0.87761832]
 [0.49029718 0.8217928  0.54518335 0.67339858]]
[[0.57108605 0.69312446 0.56395151]]
[<function identity at 0x000001A47F726660>, <function identity at 0x000001A47F726660>, <function identity at 0x000001A47F726660>, <function identity at 0x000001A47F726660>, <function identity at 0x000001A47F726660>]
[<function identity at 0x000001A47F726660>, <function identity at 0x000001A47F726660>, <function i

In [27]:
#error messages testing
print("Test for: not supported input for weights")
try:
    dim_layer = "[5,5,3,2,1]"
    testNet = FNN(dim_layer,identity,method_ini="Random")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: array of not supported dimension")
try:
    dim_layer = np.array([[5,5,3,2,1],[5,5,3,2,1]])
    testNet = FNN(dim_layer,identity,method_ini="Random")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: given numbers are not integers")
try:
    dim_layer = np.array([5,5,3.1,2,1])
    testNet = FNN(dim_layer,identity,method_ini="Random")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not enough numbers given. Minimally required input and output layers cannot be created")
try:
    dim_layer = np.array([5])
    testNet = FNN(dim_layer,identity,method_ini="Random")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Test for: not supported input for weights
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. 
Test for: array of not supported dimension
Correct errors was caught. Error:  Given array is of size not supported by implementation. Supported dimensions: 1.
Test for: given numbers are not integers
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. Given values in vector must be integers to properly represent number of neurons in layers
Test for: not enough numbers given. Minimally required input and output layers cannot be created
Correct errors was caught. Error:  Given input is not supported by implementation for weights initialization. At least two numbers are needed for FNN creation: FNN needs at least input and output layer.
